# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234223084759                   -0.50    7.0
  2   -7.250206203033       -1.80       -1.41    1.0
  3   -7.251273897097       -2.97       -2.00    4.0
  4   -7.251207975968   +   -4.18       -2.09    5.0
  5   -7.251314623016       -3.97       -2.47    4.0
  6   -7.251336729670       -4.66       -2.96    3.0
  7   -7.251338790660       -5.69       -4.12    2.0
  8   -7.251338794533       -8.41       -4.23    5.0
  9   -7.251338798638       -8.39       -5.00    3.0
 10   -7.251338798695      -10.24       -5.45    3.0
 11   -7.251338798703      -11.10       -5.91    3.0
 12   -7.251338798705      -11.79       -6.62    3.0
 13   -7.251338798705      -13.60       -7.16    3.0
 14   -7.251338798705      -14.45       -7.53    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06725796423227114
[ Info: Arnoldi iteration step 2: normres = 0.2973669262599383
[ Info: Arnoldi iteration step 3: normres = 0.9927104163572079
[ Info: Arnoldi iteration step 4: normres = 0.41651128329937154
[ Info: Arnoldi iteration step 5: normres = 0.5958072008772299
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.16e-01, 9.54e-02, 5.51e-01, 1.70e-01, 6.45e-03)
[ Info: Arnoldi iteration step 6: normres = 0.2104137256561142
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.80e-02, 9.09e-02, 1.14e-01, 1.26e-01, 2.69e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08697213569648966
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.28e-04, 6.69e-03, 8.35e-03, 4.85e-02, 6.40e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1010929156621539
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.15e-0